In [1]:
from math import pi

import pandas as pd

from bokeh.io import show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.palettes import viridis
from bokeh.io import output_notebook
output_notebook()
import picaso.justplotit as jpi

import picaso.picaso_data as ptest
import picaso.disort_disco as dtest

import numpy as np

Loading BokehJS ...

/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/crooney/anaconda3/envs/conda_venv/lib/python3.7/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /Users/crooney/Documents/codes/all-data/stellar/grp/hst/cdbs/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))


In [2]:
# Toon quadrature
Toon_quad = ptest.picaso_albedos(output_dir='Toon_quad.csv', delta_eddington=True, disort_data=False)
# Toon eddington
Toon_edd = ptest.picaso_albedos(output_dir= 'Toon_eddd.csv',Toon_coefficients ="eddington", delta_eddington=True)
# SH 2-stream
SH2 = ptest.picaso_albedos(output_dir='SH2.csv',method="SH", delta_eddington=True)
# SH 4-stream
SH4 = ptest.picaso_albedos(output_dir='SH4.csv',method='SH', stream=4, delta_eddington=True)

YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT
YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT
YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT
YOU ARE REQUESTING RESAMPLING!!
NO DATA GENERATED FOR DISORT


In [ ]:
# disort = dtest.disort_albedos(output_dir='disort_DE.csv')

In [2]:
# read in data from picaso function
data_old = pd.read_csv('DLUGACH_TEST.csv',index_col=0)
data_disort = pd.read_csv('disort_DE.csv',index_col=0)
data_SH2 = pd.read_csv('SH2.csv',index_col=0)
data_SH4 = pd.read_csv('SH4.csv',index_col=0)
data_quad = pd.read_csv('Toon_quad.csv',index_col=0)
data_edd = pd.read_csv('Toon_eddd.csv',index_col=0)

#If you want to compare two different files for the heatmap just do this
data_compare = data_old
compare_quad = (data_compare-data_quad)/data_compare*100   
compare_edd = (data_compare-data_edd)/data_compare*100   
compare_SH2 = (data_compare-data_SH2)/data_compare*100 
compare_SH4 = (data_compare-data_SH4)/data_compare*100    

quad_SH4 = (data_quad-data_SH4)/data_quad*100 
quad_SH2 = (data_quad-data_SH2)/data_quad*100 
edd_SH2 = (data_edd-data_SH2)/data_edd*100 

disort_dlugach = (data_old-data_disort)/data_old*100 

In [3]:
data=compare_quad
jpi.error_lines_plot(data)

In [4]:
jpi.error_lines_plot(compare_SH4)

In [5]:
jpi.error_lines_plot(quad_SH4)

In [ ]:
show(jpi.create_heat_map(compare_SH4, rayleigh=False))

In [ ]:
show(jpi.create_heat_map(compare_quad, rayleigh=False))

In [ ]:
show(jpi.create_heat_map(compare_edd, rayleigh=False))